In [1]:
import json
import keras_tuner as kt
import numpy as np
import pandas as pd
import plotly.express as px
import tensorflow as tf
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers

from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input

---
### Data Reading
---

In [2]:
data = np.load('../data/training_data_arg.npz', allow_pickle=True)

train_set = data['train_set']
test_set = data['test_set']
val_set = data['val_set']

In [3]:
def create_dataframe(set):
    df_ = pd.DataFrame([], columns=['x', 'y'])	
    for i in range(len(set)):
        df_ = pd.concat([df_, pd.DataFrame(
            {'reading': [set[i]['x'].reshape(24, 32)],
            'posture': set[i]['y']}
        )])
    return df_

In [4]:
df_train = create_dataframe(train_set)
df_test = create_dataframe(test_set)
df_val = create_dataframe(val_set)

print(df_train.shape, df_test.shape, df_val.shape)

(1620, 4) (54, 4) (405, 4)


In [5]:
X_train = df_train['reading']
y_train = df_train['posture']
X_val = df_val['reading']
y_val = df_val['posture']
X_test = df_test['reading']
y_test = df_test['posture']
# One-hot encode the labels
y_train = pd.get_dummies(y_train).values
y_val = pd.get_dummies(y_val).values
y_test = pd.get_dummies(y_test).values

In [6]:
X_train

0    [[24.77, 25.37, 25.45, 26.17, 26.53, 29.51, 30...
0    [[25.74, 26.52, 27.07, 28.7, 30.48, 32.82, 32....
0    [[26.5, 27.57, 28.01, 29.04, 30.02, 33.2, 33.1...
0    [[25.21, 25.82, 25.93, 26.43, 27.04, 28.96, 29...
0    [[25.85, 26.64, 27.07, 28.09, 28.72, 31.57, 32...
                           ...                        
0    [[24.078905745999865, 23.54478085637246, 24.05...
0    [[24.287305549063102, 24.46721305051553, 24.36...
0    [[23.187623444806476, 23.419400611664578, 23.4...
0    [[24.389370902565755, 24.016248933303498, 24.0...
0    [[24.309337096494875, 24.68560284513844, 24.62...
Name: reading, Length: 1620, dtype: object

In [7]:
X_train = np.array([np.array(x) for x in X_train])
X_train = np.expand_dims(X_train, -1)
X_val = np.array([np.array(x) for x in X_val])
X_val = np.expand_dims(X_val, -1)
X_test = np.array([np.array(x) for x in X_test])
X_test = np.expand_dims(X_test, -1)
y_train = np.array([np.array(x) for x in y_train])
y_test = np.array([np.array(x) for x in y_test])
y_val = np.array([np.array(x) for x in y_val])
# Convert the data to tensors
X_train = tf.convert_to_tensor(X_train)
X_val = tf.convert_to_tensor(X_val)
X_test = tf.convert_to_tensor(X_test)
y_train = tf.convert_to_tensor(y_train)
y_val = tf.convert_to_tensor(y_val)
y_test = tf.convert_to_tensor(y_test)

---
### Model Definition
---

In [8]:
model = VGG16(
    include_top=False,
    weights='imagenet',
    input_shape=(32, 32, 3),
    classes=3,
    classifier_activation="softmax",
)

Exception: URL fetch failure on https://storage.googleapis.com/tensorflow/keras-applications/vgg16/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5: None -- [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1006)

In [ ]:
early_stopping = keras.callbacks.EarlyStopping(
    patience=20,
    min_delta=0.001,
    restore_best_weights=True,
)
reduce_lr = keras.callbacks.ReduceLROnPlateau(
    patience=5
)

In [ ]:
model.compile(
    optimizer=keras.optimizers.Adam(1e-3),
    loss="categorical_crossentropy",
    metrics=["accuracy"],
)

---
### Model Training
---

In [ ]:
model.fit(
    X_train, y_train, validation_data=(X_val, y_val), batch_size=32, epochs=100, callbacks=[early_stopping, reduce_lr]
)